In [1]:
import pandas as pd
import wandb
from datetime import datetime
import numpy as np
class Results():
    def __init__(self):
        columns = [f"nguyen_{i}" for i in range(1,13,1)]
        self.results_nn_500 =  pd.DataFrame( index=list( range(1,21,1)), columns=columns)
        self.results_nn_1000 = pd.DataFrame( index=list( range(1,21,1)), columns=columns)
        self.results_grammar = pd.DataFrame( index=list( range(1,21,1)), columns=columns)
        self.results_uniform =pd.DataFrame( index=list( range(1,21,1)), columns=columns)
        

    def fill_results(self, config, summary):
 
        seed= config['seed']
        prior = config['prior_source']
        path_to_complete_model = config['path_to_complete_model']
        num_simulation_to_perfect_fit = summary['num simulation to perfect fit']
        num_states_to_perfect_fit = summary['num_states_to_perfect_fit_test']
        nguyen_number = config['data_path'].split('/')[1]

        if prior == 'uniform':
            self.results_uniform.loc[seed,nguyen_number]= num_states_to_perfect_fit

        elif prior == 'grammar':
            self.results_grammar.loc[seed,nguyen_number]= num_states_to_perfect_fit

        elif prior == 'neural_net' and path_to_complete_model=='saved_models/run_1/AlphaZero/DatasetTransformer_lin_1000/1/tf_ckpts/ckpt-138':
            self.results_nn_1000.loc[seed,nguyen_number]= num_states_to_perfect_fit

        elif prior == 'neural_net' and path_to_complete_model=='saved_models/run_1/AlphaZero/DatasetTransformer_lin_500/1/tf_ckpts/ckpt-234':
            self.results_nn_500.loc[seed,nguyen_number]= num_states_to_perfect_fit

api = wandb.Api()
entity, project = "jgu-wandb", "neural_guided_symbolic_regression"
  # Example: January 1st, 2024

#created_at (str): ISO timestamp when the run was started
runs = api.runs(entity + "/" + project, filters={"tags": {"$in": ["current_experiment"]}})

summary_list, config_list, name_list = [], [], []
results_obj = Results()

for run in runs:
    # .summary contains output keys/values for
    # metrics such as accuracy.
    #  We call ._json_dict to omit large files
    result_dict = run.summary._json_dict
    try:
        summary_dict = run.summary._json_dict
        config_dict = {k: v for k, v in run.config.items() if not k.startswith("_")}
        results_obj.fill_results(config_dict, summary_dict)
    except Exception as e:
         print(f"{e} their is a problem with {run.name} ") 

print('end')


'num simulation to perfect fit' their is a problem with neural_net__nguyen_9_500 
'num simulation to perfect fit' their is a problem with neural_net__nguyen_9_500 
'num simulation to perfect fit' their is a problem with grammar__nguyen_4 
'num simulation to perfect fit' their is a problem with grammar__nguyen_4 
'num simulation to perfect fit' their is a problem with grammar__nguyen_4 
'num simulation to perfect fit' their is a problem with grammar__nguyen_4 
'num simulation to perfect fit' their is a problem with grammar__nguyen_4 
'num simulation to perfect fit' their is a problem with neural_net__nguyen_3_500 
'num simulation to perfect fit' their is a problem with neural_net__nguyen_3_500 
'num simulation to perfect fit' their is a problem with neural_net__nguyen_3_500 
'num simulation to perfect fit' their is a problem with neural_net__nguyen_4_1000 
'num simulation to perfect fit' their is a problem with neural_net__nguyen_4_1000 
'num simulation to perfect fit' their is a proble

In [3]:
df= pd.DataFrame(results_obj.results_grammar)
df


,nguyen_1,nguyen_2,nguyen_3,nguyen_4,nguyen_5,nguyen_6,nguyen_7,nguyen_8,nguyen_9,nguyen_10,nguyen_11,nguyen_12
1,13139,18225,115056,288144,19199,NaN,5062,11553,53745,32401,14312,59239
2,5043,25526,27876,223445,3606,92638,938,18663,90008,44410,14377,16338
3,67033,13065,5933,24389,12480,107546,2758,9695,97203,21899,16843,20562
4,1907,11747,25920,101049,19520,NaN,4396,13537,71813,22310,15171,52516
5,15469,1389,9353,NaN,20774,110690,12009,12289,93450,19765,14547,34766
6,8727,17018,11942,332659,16443,NaN,9313,16673,78578,56998,14426,31730
7,7783,12674,33132,282819,3877,NaN,785,18678,223096,6141,16069,19299
8,4880,24512,15271,187475,14222,121885,6300,5732,45245,39741,13858,26022
9,3938,4455,6431,NaN,16096,114801,3933,7873,140016,59564,15960,55914
10,5861,8135,10006,31643,12394,NaN,4193,34134,188394,23541,17688,28725


In [42]:
import pandas as pd
df= pd.DataFrame(results_obj.results_nn_1000)
#df_grammar
nan_values = df.isna().sum(axis=0)
mean_values = df.mean(axis=0).round(0)
#mean_values =mean_values.astype(int)
for nguyen in list(mean_values.index): 
    print(f"{nguyen}    {mean_values.loc[nguyen]:<10} , {nan_values.loc[nguyen]}")
#df_grammar.mean(axis=0)

nguyen_1    3096.0     , 0
nguyen_2    13687.0    , 0
nguyen_3    130060.0   , 0
nguyen_4    nan        , 20
nguyen_5    1054.0     , 0
nguyen_6    97358.0    , 0
nguyen_7    10.0       , 0
nguyen_8    1194.0     , 0
nguyen_9    257961.0   , 0
nguyen_10    20315.0    , 0
nguyen_11    1315.0     , 1
nguyen_12    23120.0    , 0
